In the last notebook we saw a big problem with the forcings computed using SAM. This appeared as a meridional oscillations, which we call "zonal" striping. In this notebook, I show that these oscillations are probably caused by NGAqua's lack of mass conservation.

# Code

Here are some necesary imports and code:

In [ ]:
import holoviews as hv
hv.extension('bokeh')
%opts Image[width=500, height=300, colorbar=True](cmap='magma') 
%opts QuadMesh[width=300,  height=150, colorbar=True](cmap='viridis')
%opts Curve[width=500, height=int(500/1.61)]

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
from sam.case import get_ngqaua_ic, InitialConditionCase
import xarray as xr
from os.path import join
import numpy as np
from uwnet.thermo import layer_mass


def process_ic(ic, **kwargs):

    # open initial conditions
    case = InitialConditionCase(ic=ic)

    # configure the sam run
    case.prm['parameters'].update({
        'dosgs': False,
        'dowally': True,
        'nstop': 2,
        'nsave3d': 1,
        'nstat': 2,
        'nstatfrq': 1,
        'dt': 30
    })
    
    case.prm['parameters'].update(kwargs)

    case.save()

    # Run the model
    print(case.path)
    !cd {case.path} &&  bash -c './run.sh > out  2> err'
    # !cd {case.path} &&  bash -c './run.sh'

    # open the 3d outputs
    sam_processed = xr.open_mfdataset(join(case.path, 'OUT_3D', '*.nc'), concat_dim='time').load()
    assert len(sam_processed.time) > 1

    # clean up the working directory
    !rm -rf {case.path}
    
    return sam_processed

# Load and Process Data

Here I load the initial data from NGAqua, and use it to initialize a SAM simulation. As part of my initialization code in SAM, I call the pressure solver to correct the velocities before any physics occur.

In [ ]:
NGROOT = "/Users/noah/Data/2018-05-30-NG_5120x2560x34_4km_10s_QOBS_EQX/"
STATFILE = join(NGROOT, 'stat.nc')

stat = xr.open_mfdataset(STATFILE)
rho = stat.RHO[0]

# layer mass
dm = layer_mass(rho)

# get the initial condition at first time step
# the winds are staggered
ic = get_ngqaua_ic(NGROOT, 0)

# Process this initial conditions using SAM
sam_processed = process_ic(ic, dt=.01).assign_coords(time=np.arange(3))

What does the pressure in the first time step look like? If the initial data is divergent free, this pressure should vanish for the "0th" output if the initial data conserve mass.

In [ ]:
hv.Dataset(sam_processed.PP[:,::5]).to.image(["x", "y"])

What does the zonal mean of this pressure correcting velocity look like?

In [ ]:
(sam_processed.PP.mean('x')[0,::5]/rho).plot()
plt.title(r"$-p'/\rho_0$ (Pa)")

The pressure is constant in height. What is the corresponding adjustment to $V$.

In [ ]:
v_p = sam_processed.V - ic.V.assign_coords(x=sam_processed.x, y=sam_processed.y)
v_p.name = 'V - Init cond'
v_p.mean('x')[0].plot()

**The difference in velocity between the initial conditions and the pressure-corrected data is almost perfectly constant in height.** How could this happen? The zonal and vertical average of mass conservations implies that $<\bar{V}>_y = 0$. However, this is probably not satisfied for this initial data, since we are skipping many grid-cells.

In [ ]:
v_barotropic = (ic.V.mean('x') * dm).sum('z')/dm.sum()
v_barotropic.plot()
plt.title("Zonal/Vertical Average of V");

Indeed, there is a large fluctuation of the zonal/barotropic average $V$. Is there a correspondingly large deviation in the zonal velocity?

In [ ]:
u_barotropic = (ic.U.mean('y') * dm).sum('z')/dm.sum()
hv.Curve(u_barotropic, vdims=['U']).redim.unit(U='m/s')

In [ ]:
plt.rcParams['axes.formatter.useoffset'] = False
u_barotropic.plot()

$U$ flucutuates a similar amount about its mean value as $V$ does. However, the mean value need not vanish, so the percent fluctuations are much smaller.


These should both be constant. That they aren't most likely means that I am using the wrong layer mass to compute the meridional divergence. Luckily, the true density can be backwards engineered by solving a linear system for rho. For each latitude $i$, $\sum_k\bar{V}_k^i m_k =0$ where $m_k$ is the layer mass of level $k$. Because there are 64 latitudes, and only 34 vertical levels, I can solve a linear system to find $m$. The matrix to do this is $A_{ij} = \bar{V}_{ij}$. So $Am=0$ is the linear system to solve. To do this, we need to find the null space of the matrix $A$, which we can easily do using the SVD.

In [ ]:
from scipy.linalg import svd, null_space

A = np.vstack([ic.V.mean('x').T.values, ic.U.mean('y').diff('x').T.values])

U, S, Vt = svd(A, full_matrices=False)
m = Vt[-1] * Vt[-1][0]

In [ ]:
plt.plot(m)
plt.title(f"Last singular vector, Singular value = {S[-1]:.2e}, max sing val={S[0]:.2e}")

this looks a lot like the layer mass.

In [ ]:
plt.plot(dm)

## Vertical velocity

In a previous notebook, I showed the domain average vertical velocity in NGAqua does not vanish:

In [ ]:
ic.W.mean(['x', 'y']).plot(y='z')

# Conclusions

The NG-Aqua data does not seem to conserve mass. This appeared in a previous notebook when the domain averaged vertical velocity did not vanish, and in this context there is net mass transport through circles of latitude. These problems do not harm the advection forcing much if computed using the advection form $\mathbf v \cdot\nabla f$, but in divergence form these problems can wreak havoc.